# Assignment 1: KPD Optimization Problems

The purpose of this assignment is to gain a deeper understanding of and hands-on experience with the
optimization problems that arise in KPD

## Step 1: Data Preparation

In [8]:
import random
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import matplotlib.pyplot as plt

## Step 1: Data Preparation

In [9]:
seed = 3939
random.seed(seed)
np.random.seed(seed)

N = random.randint(20, 25)

A1 = np.array([random.sample(range(1, N + 1), N) for _ in range(N)])

A2 = np.random.choice([0, 1], size=(N, N), p=[0.5, 0.5])

In [10]:
# Number of patient-donor pairs (N) as an integer between 20 and 25
N

22

In [11]:
A1

array([[14, 13,  2, 10, 15,  5,  6, 18, 22, 20, 16, 11,  7, 12,  1, 19,
        17, 21,  8,  3,  9,  4],
       [14, 10, 20,  6, 22,  5,  2, 18, 17,  7,  4,  9,  3, 21, 19, 16,
        15, 11, 13, 12,  1,  8],
       [15,  5, 13, 20, 12, 19,  3,  8, 16,  6, 21,  7, 14, 17,  4,  2,
        18,  1, 22, 11,  9, 10],
       [ 9, 12, 18,  8, 20, 15, 21,  6, 16,  1, 19,  3, 22,  4,  7,  5,
        13, 17, 10, 11, 14,  2],
       [ 2, 11,  5,  6, 12, 18, 10,  1,  8, 15, 17, 21,  7,  3, 13, 22,
        20,  4, 14, 19,  9, 16],
       [20, 15,  9, 13,  1,  8, 17,  6, 21, 22,  3, 18,  4, 14, 11, 19,
         2,  5,  7, 12, 10, 16],
       [ 8, 12,  6, 19, 15,  9,  1, 18, 17, 14, 20, 13, 10,  7,  3, 16,
         2, 22,  4,  5, 21, 11],
       [17, 11,  8, 20, 14, 13,  1, 22, 12,  9,  6,  7,  3, 16, 15, 18,
        21,  2,  5, 10,  4, 19],
       [ 1, 18,  7,  4,  8,  2,  6,  5, 12,  9, 13, 15, 19, 22, 16, 10,
        11, 14, 20,  3, 21, 17],
       [20, 12, 18,  8,  2, 22,  1,  5, 15, 11,  6, 10,

In [ ]:
A2

array([[0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1],
       [1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1],
       [0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0],
       [0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
       [1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1],
       [0, 0, 0, 1, 0, 0,

## Part 1.2 - Optimization Problems
### A. Minimizing Total Preference

In [13]:
N = len(A1)
model = gp.Model("Minimize_Total_Preference")
x = model.addVars(N, N, vtype=GRB.BINARY, name="x")

# Objective
model.setObjective(gp.quicksum(A1[i, j] * x[i, j] for i in range(N) for j in range(N)), GRB.MINIMIZE)

# Constraints
model.addConstrs((gp.quicksum(x[i, j] for j in range(N)) == 1 for i in range(N)), name="Patient_Receive")
model.addConstrs((gp.quicksum(x[i, j] for i in range(N)) == 1 for j in range(N)), name="Donor_Donate")

model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23H222)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 44 rows, 484 columns and 968 nonzeros
Model fingerprint: 0x658ae1ca
Variable types: 0 continuous, 484 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 243.0000000
Presolve time: 0.00s
Presolved: 44 rows, 484 columns, 968 nonzeros
Variable types: 0 continuous, 484 integer (484 binary)

Root relaxation: objective 3.400000e+01, 38 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      34.0000000   34.00000  0.00%     -    0s

Explored 1 nodes (38 

### A. Minimizing Maximum Preference

In [ ]:
model = gp.Model("Minimize_Maximum_Preference")

x = model.addVars(N, N, vtype=GRB.BINARY, name="x")  # binary assignment variable
z = model.addVar(vtype=GRB.INTEGER, name="z")  # max preference variable

model.setObjective(z, GRB.MINIMIZE)

model.addConstrs((gp.quicksum(x[i, j] for j in range(N)) == 1 for i in range(N)), name="Patient_Receive")

model.addConstrs((gp.quicksum(x[i, j] for i in range(N)) == 1 for j in range(N)), name="Donor_Donate")

model.addConstrs((A1[i, j] * x[i, j] <= z for i in range(N) for j in range(N)), name="Max_Preference")

model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"\nOptimal Maximum Preference: {model.objVal}\n")
    solution = [(i + 1, j + 1, A1[i, j]) for i in range(N) for j in range(N) if x[i, j].x > 0.5]
    print("Selected Matches (Patient -> Donor) and Preferences:")
    for match in solution:
        print(f"Patient {match[0]} -> Donor {match[1]}, Preference Score: {match[2]}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23H222)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 528 rows, 485 columns and 1936 nonzeros
Model fingerprint: 0x15bca728
Variable types: 0 continuous, 485 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 22.0000000
Presolve removed 461 rows and 0 columns
Presolve time: 0.01s
Presolved: 67 rows, 485 columns, 1453 nonzeros
Variable types: 0 continuous, 485 integer (484 binary)

Root relaxation: objective 2.875000e+00, 97 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.87500    0   27   22.00000    2.87500

## Part 1.3. 2-Way Exchanges

In [ ]:

model = gp.Model("2-Way Kidney Exchange")

x = model.addVars(N, N, vtype=GRB.BINARY, name="x")  # Binary assignment variable

model.setObjective(gp.quicksum(x[i, j] for i in range(N) for j in range(N)), GRB.MAXIMIZE)

model.addConstrs((gp.quicksum(x[i, j] for j in range(N)) <= 1 for i in range(N)), name="Patient_Receive")

model.addConstrs((gp.quicksum(x[i, j] for i in range(N)) <= 1 for j in range(N)), name="Donor_Donate")

model.addConstrs((x[i, j] + x[j, i] <= 1 for i in range(N) for j in range(N) if i != j), name="Two_Way_Exchange")

model.addConstrs((x[i, j] <= A2[i, j] for i in range(N) for j in range(N)), name="Compatibility")

model.optimize()

if model.status == GRB.OPTIMAL:
    print(f"\nTotal Successful 2-Way Exchanges: {model.objVal}\n")
    solution = [(i + 1, j + 1) for i in range(N) for j in range(N) if x[i, j].x > 0.5]
    print("Selected Matches (Patient -> Donor):")
    for match in solution:
        print(f"Patient {match[0]} -> Donor {match[1]}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23H222)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 990 rows, 484 columns and 2376 nonzeros
Model fingerprint: 0xa9c8986a
Variable types: 0 continuous, 484 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 21.0000000
Presolve removed 885 rows and 258 columns
Presolve time: 0.00s
Presolved: 105 rows, 226 columns, 620 nonzeros
Variable types: 0 continuous, 226 integer (226 binary)

Root relaxation: objective 2.200000e+01, 47 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      22.0000000   22.000

## Part 2.1. Additional Priority and Compatibility Restrictions
### Restricting  the exchange to 2-way swaps and applying additional constraints based on priority order and preference thresholds

In [24]:
    N = len(A2)

    model = gp.Model("2-Way_Exchange_Priority")
    
    x = model.addVars(N, N, vtype=GRB.BINARY, name="x")
    
    model.addConstrs((gp.quicksum(x[i, j] for j in range(N)) <= 1 for i in range(N)), name="Patient_Receive")
    model.addConstrs((gp.quicksum(x[j, i] for j in range(N)) <= 1 for i in range(N)), name="Donor_Donate")

    model.addConstrs((x[i, j] == x[j, i] for i in range(N) for j in range(N) if i != j), name="Two_Way_Exchange")

    model.addConstrs((x[i, j] == 0 for i in range(N) for j in range(N) if A1[i, j] >= 11), name="Preference_Threshold")

    model.setObjective(gp.quicksum((N - i) * x[i, j] for i in range(N) for j in range(N)), GRB.MAXIMIZE)

    model.optimize()

    assignment = np.zeros((N, N))
    if model.status == GRB.OPTIMAL:
        for i in range(N):
            for j in range(N):
                assignment[i, j] = x[i, j].X  
    else:
        print("No optimal solution found.")

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23H222)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 770 rows, 484 columns and 2156 nonzeros
Model fingerprint: 0xd1d8a1f7
Variable types: 0 continuous, 484 integer (484 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 110.0000000
Presolve removed 750 rows and 448 columns
Presolve time: 0.00s
Presolved: 20 rows, 36 columns, 73 nonzeros
Found heuristic solution: objective 133.0000000
Variable types: 0 continuous, 36 integer (36 binary)
Found heuristic solution: objective 252.0000000

Root relaxation: objective 2.530000e+02, 15 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc